In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os
import sys

if os.getcwd().split('/')[-1] == 'notebooks':
    os.chdir('../')
os.getcwd()

'/home/lehl/development/QueryOptDeepDive'

In [3]:
import os
import re
import datetime
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
from mo_sql_parsing import parse

In [5]:
aliases = set()

for file in os.listdir('balsa/queries/stack/'):
    if not file.endswith('.sql'):
        continue

    with open(os.path.join('balsa', 'queries', 'stack', file), 'r') as f:
        lines = f.readlines()

    sql = ''.join(lines)
    res = parse(sql)

    for alias in res['from']:
        if isinstance(alias, str):
            aliases.add(alias)
        else:
            aliases.add(alias['name'])

aliases

{'a1',
 'acc',
 'account',
 'b',
 'b1',
 'b2',
 'c1',
 'c2',
 'pl',
 'q',
 'q1',
 'q2',
 'question',
 's',
 's1',
 's2',
 'site',
 'so_user',
 't',
 't1',
 't2',
 'tag',
 'tag_question',
 'tq',
 'tq1',
 'tq2',
 'u1',
 'u2'}

In [8]:
aliases = sorted(aliases)

id2aliasname = dict()
aliasname2id = dict()

id2aliasname[0] = 'start'
aliasname2id['start'] = 0

for aid, alias in enumerate(aliases):
    id2aliasname[aid+1] = alias
    aliasname2id[alias] = aid+1

print(f"id2aliasname = {id2aliasname}")
print()
print(f"aliasname2id = {aliasname2id}")

id2aliasname = {0: 'start', 1: 'a1', 2: 'acc', 3: 'account', 4: 'b', 5: 'b1', 6: 'b2', 7: 'c1', 8: 'c2', 9: 'pl', 10: 'q', 11: 'q1', 12: 'q2', 13: 'question', 14: 's', 15: 's1', 16: 's2', 17: 'site', 18: 'so_user', 19: 't', 20: 't1', 21: 't2', 22: 'tag', 23: 'tag_question', 24: 'tq', 25: 'tq1', 26: 'tq2', 27: 'u1', 28: 'u2'}

aliasname2id = {'start': 0, 'a1': 1, 'acc': 2, 'account': 3, 'b': 4, 'b1': 5, 'b2': 6, 'c1': 7, 'c2': 8, 'pl': 9, 'q': 10, 'q1': 11, 'q2': 12, 'question': 13, 's': 14, 's1': 15, 's2': 16, 'site': 17, 'so_user': 18, 't': 19, 't1': 20, 't2': 21, 'tag': 22, 'tag_question': 23, 'tq': 24, 'tq1': 25, 'tq2': 26, 'u1': 27, 'u2': 28}


In [9]:
id2aliasname = {
    0: 'start', 1: 'a1', 2: 'acc', 3: 'account', 4: 'b', 5: 'b1', 6: 'b2', 7: 'c1', 8: 'c2', 9:
    'pl', 10: 'q', 11: 'q1', 12: 'q2', 13: 'question', 14: 's', 15: 's1', 16: 's2', 17: 'site', 18: 'so_user', 19: 't',
    20: 't1', 21: 't2', 22: 'tag', 23: 'tag_question', 24: 'tq', 25: 'tq1', 26: 'tq2', 27: 'u1', 28: 'u2'
}

aliasname2id = {
    'start': 0, 'a1': 1, 'acc': 2, 'account': 3, 'b': 4, 'b1': 5, 'b2': 6, 'c1': 7, 'c2': 8, 'pl': 9,
    'q': 10, 'q1': 11, 'q2': 12, 'question': 13, 's': 14, 's1': 15, 's2': 16, 'site': 17, 'so_user': 18,
    't': 19, 't1': 20, 't2': 21, 'tag': 22, 'tag_question': 23, 'tq': 24, 'tq1': 25, 'tq2': 26, 'u1': 27, 'u2': 28
}

len(id2aliasname.keys()), len(aliasname2id.keys())

(29, 29)

In [10]:
all_queries = os.listdir('balsa/queries/stack')
all_queries = [q for q in all_queries if q.endswith('sql')]
print(all_queries)

['q5__q5-052.sql', 'q3__q3-068.sql', 'q3__q3-046.sql', 'q5__q5-041.sql', 'q4__q4-002.sql', 'q8__q8-096.sql', 'q4__q4-086.sql', 'q16__ea9efde510227beb8d624b8c4a6941b9d5e6e637.sql', 'q7__q7-034.sql', 'q16__fbe34e8fdf672a34fd82cbbd6d9a81fd02ce17d1.sql', 'q14__5dbc1d1f1a0467ad0086e6cb337144387a37533a.sql', 'q6__q6-067.sql', 'q12__55de941e8497cfeeb93d3f8f2d7a18489e0e6c32.sql', 'q6__q6-064.sql', 'q1__q1-067.sql', 'q4__q4-064.sql', 'q3__q3-066.sql', 'q13__a3d03772d880754fc4e150d82908757477ae2186.sql', 'q13__d383cd5b4aee7d3f73508e2a1fe5f6d0f7dd42a2.sql', 'q5__q5-077.sql', 'q5__q5-032.sql', 'q15__3e37e62655ceaebc14e79edad518e5710752f51d.sql', 'q11__c1ae2a992cde4ea2c4922d852df22043254b4f84.sql', 'q14__74fd1af68d23f0690e3d0fc80bd9b42fa90a7e94.sql', 'q16__ed2ffeaefcf5ad8bbadc713ccc766541e12080aa.sql', 'q1__q1-100.sql', 'q12__06c8d6886a03d4d92837f38ff395b888de007d33.sql', 'q7__q7-085.sql', 'q4__q4-026.sql', 'q14__719e692d411868ae7a93909757872d264f6bbf73.sql', 'q14__b49361f85785200ed6ec1f2eec357b759

In [18]:
QUERY_TIMEOUT = 2 * 3 * 60 * 1000

In [20]:
TEST_QUERIES = dict()

for folder in os.listdir('bao/queries'):
    if not folder.startswith('stack__'):
        continue
        
    key = folder.split('stack__')[1]
    
    queries = os.listdir(os.path.join('bao', 'queries', folder, 'test'))
    queries = sorted(queries)

    TEST_QUERIES[key] = queries
    
for k in sorted(TEST_QUERIES.keys()):
    print(k)
    print(TEST_QUERIES[k])
    print('---' * 30)

# TEST_QUERIES = {
#     'base_query_split_1': ['02a', '02b', '02c', '02d', '07a', '07b', '07c', '15a', '15b', '15c', '15d', '24a', '24b', '25a', '25b', '25c', '31a', '31b', '31c'],
#     'base_query_split_2': ['13a', '13b', '13c', '13d', '15a', '15b', '15c', '15d', '20a', '20b', '20c', '26a', '26b', '26c', '29a', '29b', '29c', '30a', '30b', '30c', '33a', '33b', '33c'],
#     'base_query_split_3': ['01a', '01b', '01c', '01d', '05a', '05b', '05c', '12a', '12b', '12c', '17a', '17b', '17c', '17d', '17e', '17f', '22a', '22b', '22c', '22d', '27a', '27b', '27c', '28a', '28b', '28c'],
#     'leave_one_out_split_1': ['01c', '02a', '03b', '04a', '05a', '06b', '07c', '08c', '09c', '10b', '11b', '12c', '13b', '14a', '15b', '16c', '17c', '18b', '19a', '20c', '21c', '22b', '23b', '24a', '25a', '26c', '27c', '28a', '29b', '30a', '31b', '32b', '33c'],
#     'leave_one_out_split_2': ['01d', '02d', '03a', '04b', '05c', '06d', '07a', '08c', '09c', '10a', '11a', '12a', '13d', '14b', '15b', '16a', '17f', '18a', '19d', '20a', '21b', '22c', '23b', '24b', '25a', '26a', '27b', '28c', '29a', '30b', '31a', '32b', '33b'],
#     'leave_one_out_split_3': ['01c', '02d', '03b', '04a', '05c', '06d', '07b', '08a', '09a', '10c', '11d', '12a', '13a', '14b', '15a', '16d', '17b', '18b', '19d', '20b', '21a', '22a', '23b', '24a', '25b', '26a', '27a', '28b', '29c', '30a', '31a', '32a', '33c'],
#     'random_split_1': ['01c', '02c', '04b', '04c', '05c', '06a', '06c', '06e', '08b', '08c', '09c', '11d', '15a', '17b', '17e', '18b', '20a', '21a', '25c', '28b', '32b', '33a'],
#     'random_split_2': ['01a', '04c', '05c', '06c', '06d', '07b', '08c', '10a', '11a', '11d', '13c', '13d', '15d', '16a', '17b', '19a', '20a', '22b', '25b', '29b', '31a', '32b'],
#     'random_split_3': ['02a', '03b', '06d', '09b', '10b', '11b', '11c', '13c', '13d', '16b', '18c', '19c', '21c', '22a', '22d', '26a', '26b', '27c', '28a', '28c', '30a', '33c'],
# }

base_query_split_1
['q13__13ad1b8c6bea4fda1892b9fa82cc1ceb9ceb85fc.sql', 'q13__1ddcc8650e17b292bc7344902baffc90c5ae5761.sql', 'q13__935e2051bf80eeafe91aeb6eb719b6b64b9592c2.sql', 'q13__a091adce62743b65c04532e98e8ff3d7e546ea77.sql', 'q13__a3d03772d880754fc4e150d82908757477ae2186.sql', 'q13__add0df9dccb2790c14508e19c9e0deb79fad6ea2.sql', 'q13__d383cd5b4aee7d3f73508e2a1fe5f6d0f7dd42a2.sql', 'q13__d4707be2adfdbc842f42acb1fc16e3a43faf7474.sql', 'q2__q2-001.sql', 'q2__q2-012.sql', 'q2__q2-032.sql', 'q2__q2-035.sql', 'q2__q2-050.sql', 'q2__q2-081.sql', 'q2__q2-094.sql', 'q2__q2-098.sql', 'q7__q7-034.sql', 'q7__q7-036.sql', 'q7__q7-047.sql', 'q7__q7-077.sql', 'q7__q7-082.sql', 'q7__q7-085.sql', 'q7__q7-095.sql', 'q7__q7-099.sql']
------------------------------------------------------------------------------------------
leave_one_out_split_1
['q11__6c5cba419c5b7b02d431aeb5e766d775d812967a.sql', 'q12__547c6bf1994c9b2ba82a7ae32f4b051beabf46fd.sql', 'q13__935e2051bf80eeafe91aeb6eb719b6b64b9592c2.s

In [21]:
def build_workload(queries):
    tmp = []

    for q in queries:
        with open(os.path.join('balsa', 'queries', 'stack', q), 'r') as f:
            lines = f.readlines()

        sql = ''.join(lines)
        sql = sql.split(';')[0] + ';'
            
        tmp.append([sql, q, [-1, False]])
        
    return tmp

In [27]:
for key in TEST_QUERIES.keys():
    base_path = f"hybrid_qo/workload/STACK__{key}"
    print(f'WORKLOAD: {key} ({base_path})')
    #os.remove(f"{base_path}__train.json")
    #os.remove(f"{base_path}__test.json")
    
    test_queries = []
    for q in TEST_QUERIES[key]:
        test_queries.append(q)
    
    train_queries = []
    for q in sorted(all_queries):
        if q not in test_queries:
            train_queries.append(q)
    
    
    train_wl = build_workload(train_queries)
    with open(f"{base_path}__train.json", 'w') as f:
        json.dump(train_wl, f)
    
    test_wl = build_workload(test_queries)
    with open(f"{base_path}__test.json", 'w') as f:
        json.dump(test_wl, f)
    


WORKLOAD: base_query_split_1 (hybrid_qo/workload/STACK__base_query_split_1)
WORKLOAD: leave_one_out_split_1 (hybrid_qo/workload/STACK__leave_one_out_split_1)
WORKLOAD: random_split_1 (hybrid_qo/workload/STACK__random_split_1)


In [28]:
sorted(os.listdir('hybrid_qo/workload'))

['JOB__base_query_split_1__test.json',
 'JOB__base_query_split_1__train.json',
 'JOB__base_query_split_2__test.json',
 'JOB__base_query_split_2__train.json',
 'JOB__base_query_split_3__test.json',
 'JOB__base_query_split_3__train.json',
 'JOB__leave_one_out_split_1__test.json',
 'JOB__leave_one_out_split_1__train.json',
 'JOB__leave_one_out_split_2__test.json',
 'JOB__leave_one_out_split_2__train.json',
 'JOB__leave_one_out_split_3__test.json',
 'JOB__leave_one_out_split_3__train.json',
 'JOB__random_split_1__test.json',
 'JOB__random_split_1__train.json',
 'JOB__random_split_2__test.json',
 'JOB__random_split_2__train.json',
 'JOB__random_split_3__test.json',
 'JOB__random_split_3__train.json',
 'JOB_dynamic.json',
 'JOB_static.json',
 'STACK__base_query_split_1__test.json',
 'STACK__base_query_split_1__train.json',
 'STACK__leave_one_out_split_1__test.json',
 'STACK__leave_one_out_split_1__train.json',
 'STACK__random_split_1__test.json',
 'STACK__random_split_1__train.json']